In [9]:
import os
import glob
import pandas as pd
import re

In [10]:
conjunto_dados = pd.DataFrame()

SPEAKER_REGEX = re.compile(r'[A|O] Sr\.ª?( [\w\-,]+)+([\-\w]+)?( \([\w\s\.,\-]+\))*(?=:)')
MULTILINE_SPEAKER_REGEX = re.compile("^" + SPEAKER_REGEX.pattern, flags=re.M) 

for file in os.scandir("./discursos"):
    speech_by_speaker = []

    with open(file) as fp:
        content = fp.read()

        #1) remover datas espalhadas, páginas e afins espalhadas pelos documentos (por causa da coversao PDF para TXT)

        datas_texto = re.compile("\d{1,2} [A-Z]+ [A-Z]+ [A-Z]+ \d{4} (\n\n\d+)?")
        datas_texto.sub(" ", content)

        series_numeros = re.compile("[A-Z]+ SÉRIE \W NÚMERO \d+ (\n+\d+)?")
        series_numeros.sub(" ", content)

        #2) criar dicionario de intervencoes para criar dataframe

        #2.1) identificar os intervenientes e intervencoes correspondentes

        deputadxs = MULTILINE_SPEAKER_REGEX.finditer(content) 
        deputadxs = [match.group() for match in deputadxs]
        deputadxs = list(set(deputadxs))

        #deputadxs = [deputadxs[5]]
        #print(deputadxs)

        content_on_one_line = ''.join(content.splitlines())

        for speaker in deputadxs:
            content_split_by_speaker = content_on_one_line.split(speaker + ':')
            # Get rid of everything before they speak
            content_split_by_speaker.pop(0)

            # what_they_say = [speech for speech in speaker_regex]
            # for each line in content_split_by_speaker
            # use the speaker regex to find the next speaker and get everything before that
            for line in content_split_by_speaker:
                start_of_next_speaker = SPEAKER_REGEX.finditer(line)

                # either an index or None
                # if it's None, either:
                # - the regex isn't accurate enough; or
                # - the speaker is the only person to speak]
                next_speaker_indexes = [match.start() for match in start_of_next_speaker]
                next_speaker_index = next_speaker_indexes[0] if len(next_speaker_indexes) > 0 else None

                speech = line[0:next_speaker_index] if next_speaker_index is not None else line
                speech_by_speaker.append((speaker, speech))

                # if "):" in speech:
                #     print(speech)
        
        df = pd.DataFrame.from_records(speech_by_speaker, columns=["speaker", "speech"])#.reset_index()
        df['filename'] = os.path.basename(file)
        #print(df)
                
        conjunto_dados = pd.concat([conjunto_dados, df])
            
conjunto_dados#.to_csv("deputados.csv")#, index=False)
    

,speaker,speech,filename
0,A Sr.ª Emília Cerqueira (PSD),"— Sr. Presidente, Srs. Deputados, saúdo Os Ve...",darl13sl03n050.txt
1,A Sr.ª Emília Cerqueira (PSD),"— Portanto, estamos a falar, infelizmente, de...",darl13sl03n050.txt
2,A Sr.ª Emília Cerqueira (PSD),"— Terminarei, Sr. Presidente, perguntando a O...",darl13sl03n050.txt
3,A Sr.ª Emília Cerqueira (PSD),— Não é verdade!,darl13sl03n050.txt
4,A Sr.ª Emília Cerqueira (PSD),"— Sr. Presidente, Sr. Deputado Pedro Soares, ...",darl13sl03n050.txt
...,...,...,...
229,A Sr.ª Heloísa Apolónia (Os Verdes),"— Sr. Presidente, Sr.as e Srs. Deputados: O p...",darl13sl03n058.txt
230,A Sr.ª Heloísa Apolónia (Os Verdes),"— É por estas razões e por tantas outras, Sr....",darl13sl03n058.txt
231,O Sr. António Filipe (PCP),"— Sr. Presidente, Sr.ª Ministra, Sr.ª Secretá...",darl13sl03n058.txt
232,O Sr. António Filipe (PCP),"— Sr. Presidente, Srs. Membros do Governo, Sr...",darl13sl03n058.txt


In [11]:
conjunto_dados["speech"] = [string.lstrip("— ") for string in conjunto_dados["speech"]]


partidos_raw = conjunto_dados["speaker"].str.split("(", n=1, expand=True)
partidos_raw.columns = ["speaker", "partido"]
partidos_raw["partido"] = partidos_raw["partido"].str.rstrip(")")
partidos_raw["speech"] = conjunto_dados["speech"]
partidos_raw["filename"] = conjunto_dados["filename"]
partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")
partidos_raw["speaker"] = partidos_raw["speaker"].str.rstrip(" ")
#conjunto_dados = conjunto_dados.merge(partidos_raw, how="right", on="speaker") it was breaking (maybe memory issues?)

partidos_raw.to_csv("deputados.csv", index=False)

/tmp/ipykernel_11774/3648350944.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")


In [12]:
partidos_raw

,speaker,partido,speech,filename
0,Emília Cerqueira,PSD,"Sr. Presidente, Srs. Deputados, saúdo Os Verde...",darl13sl03n050.txt
1,Emília Cerqueira,PSD,"Portanto, estamos a falar, infelizmente, de um...",darl13sl03n050.txt
2,Emília Cerqueira,PSD,"Terminarei, Sr. Presidente, perguntando a Os V...",darl13sl03n050.txt
3,Emília Cerqueira,PSD,Não é verdade!,darl13sl03n050.txt
4,Emília Cerqueira,PSD,"Sr. Presidente, Sr. Deputado Pedro Soares, hoj...",darl13sl03n050.txt
...,...,...,...,...
229,Heloísa Apolónia,Os Verdes,"Sr. Presidente, Sr.as e Srs. Deputados: O prob...",darl13sl03n058.txt
230,Heloísa Apolónia,Os Verdes,"É por estas razões e por tantas outras, Sr. Pr...",darl13sl03n058.txt
231,António Filipe,PCP,"Sr. Presidente, Sr.ª Ministra, Sr.ª Secretária...",darl13sl03n058.txt
232,António Filipe,PCP,"Sr. Presidente, Srs. Membros do Governo, Srs. ...",darl13sl03n058.txt


In [13]:
print(partidos_raw["partido"].unique())

['PSD' 'BE' 'CDS-PP' 'PS' 'PCP' 'Emília Santos' 'Pedro Alves'
 'Jorge Lacão' None 'Os Verdes' 'André Moz Caldas' 'António Filipe' 'PEV'
 'Cláudia Pereira' 'CH' 'PAN' 'Mariana Vieira da Silva'
 'Maria da Luz Rosinha' 'IL' 'Rosa Monteiro' 'Eduardo Cabrita'
 'Diogo Leão' 'Marta Temido' 'José de Matos Correia'
 'António Carlos Monteiro' 'Duarte Pacheco' 'N insc.' 'Gabriel Bastos'
 'Sofia Araújo' 'Mário Belo Morgado' 'Duarte Cordeiro' 'Fernando Negrão'
 'Jorge Delgado' 'Helga Correia' 'João Leão' 'António Mendonça Mendes'
 'João Galamba' 'Mário Centeno' 'António Costa' 'Ana Mesquita'
 'Helena Mesquita Ribeiro' 'Nelson de Souza' 'Maria de Fátima Fonseca'
 'Sandra Pontedeira' 'José Manuel Pureza' 'Ana Paula Zacarias'
 'Luís Capoulas Santos' 'Teresa Caeiro' 'Pedro Nuno Santos'
 'Miguel Cabrita' 'Augusto Santos Silva' 'João Neves' 'Alexandra Leitão'
 'Carlos Miguel' 'OS Verdes' 'Luís Filipe de Castro Mendes'
 'Ana Paula Vitorino' 'Idália Salvador Serrão' 'Os verdes'
 'Moisés Ferreira' 'Inês dos

In [14]:
#correções manuais partidos

partidos_raw.loc[partidos_raw["partido"] == "PEN", "partido"] = 'PAN'

partidos_raw.loc[partidos_raw["partido"] == "OS Verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os Vedes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os Verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "s Verdes", "partido"] = 'PEV'

partidos_raw.loc[partidos_raw["partido"] == "SD", "partido"] = 'PSD'
partidos_raw.loc[partidos_raw["partido"] == "PD", "partido"] = 'PSD'

partidos_raw.loc[partidos_raw["partido"] == "CDS", "partido"] = 'CDS-PP'
partidos_raw.loc[partidos_raw["partido"] == "CSD-PP", "partido"] = 'CDS-PP'
partidos_raw.loc[partidos_raw["partido"] == "CDS-P", "partido"] = 'CDS-PP'

partidos_raw.loc[partidos_raw["partido"] == "PC", "partido"] = 'PCP'

partidos_raw.loc[partidos_raw["partido"] == "B E", "partido"] = 'BE'
partidos_raw.loc[partidos_raw["partido"] == "Bloco de Esquerda", "partido"] = 'BE'

partidos_raw.loc[partidos_raw["partido"] == "Partido Socialista", "partido"] = 'PS'

partidos_raw.loc[partidos_raw["partido"] == "N insc.", "partido"] == "N insc"
partidos_raw.loc[partidos_raw["partido"] == "N Insc.", "partido"] == "N insc"
partidos_raw.loc[partidos_raw["partido"] == "Ninsc.", "partido"] == "N insc"
partidos_raw.loc[partidos_raw["partido"] == "N. insc.", "partido"] == "N insc"
partidos_raw.loc[partidos_raw["partido"] == "Não insc.", "partido"] == "N insc"

65    False
66    False
Name: partido, dtype: bool

In [15]:
declaracoes_partido = partidos_raw.loc[(partidos_raw["partido"] == "PS") \
                |(partidos_raw["partido"] == "PSD") \
                |(partidos_raw["partido"] == "CDS-PP") \
                |(partidos_raw["partido"] == "BE") \
                |(partidos_raw["partido"] == "PCP")\
                |(partidos_raw["partido"] == "PEV")\
                |(partidos_raw["partido"] == "PAN")\
                |(partidos_raw["partido"] == "IL") \
                |(partidos_raw["partido"] == "CH") \
                |(partidos_raw["partido"] == "L") \
                |(partidos_raw["partido"] == "N insc")]

declaracoes_partido.to_csv("so-partidos.csv", index=False)